In [ ]:
# ==============================================================================
# PASSO 0: INSTALAÇÃO DAS DEPENDÊNCIAS
# ==============================================================================
# Este bloco instala todas as bibliotecas necessárias da Hugging Face e outras
# ferramentas que usaremos para processamento de imagem e avaliação.
!pip install -q transformers datasets Pillow seqeval accelerate
!pip install -q "evaluate" # Usado pela Hugging Face para métricas

# ==============================================================================
# PASSO 1: CARREGAMENTO E PREPARAÇÃO DO DATASET DE EXEMPLO (FUNSD)
# ==============================================================================
# NOTA IMPORTANTE: Aqui usamos o dataset 'funsd' como exemplo.
# Para seu projeto real, você substituirá esta seção para carregar
# suas próprias imagens e anotações de Notas Fiscais.
print(">>> Carregando o dataset de exemplo (FUNSD)...")
from datasets import load_dataset

# Carregamos o dataset do Hugging Face Hub
dataset = load_dataset("nielsr/funsd")

# O dataset já vem dividido em treino e teste
train_dataset = dataset["train"]
eval_dataset = dataset["test"]

# Vamos inspecionar os rótulos (labels) do dataset
labels = dataset["train"].features["ner_tags"].feature.names
print(f"\nOs rótulos (entidades) neste dataset são: {labels}")

# Criamos mapeamentos de ID para rótulo e vice-versa.
# Você fará o mesmo para suas entidades (ex: 'cnpj_emitente', 'valor_total').
id2label = {i: label for i, label in enumerate(labels)}
label2id = {label: i for i, label in enumerate(labels)}

print(f"\nMapeamento id2label: {id2label}")
print(f"Mapeamento label2id: {label2id}")


# ==============================================================================
# PASSO 2: PRÉ-PROCESSAMENTO DOS DADOS
# ==============================================================================
# Esta é a etapa mais técnica, onde transformamos as imagens e textos em um
# formato que o modelo LayoutLMv3 entende.
print("\n>>> Iniciando o pré-processamento dos dados...")
from transformers import AutoProcessor
from PIL import Image

# Carregamos o processador do modelo. Ele lida com a tokenização do texto
# e o processamento da imagem (OCR implícito e normalização).
model_checkpoint = "microsoft/layoutlmv3-base"
processor = AutoProcessor.from_pretrained(model_checkpoint, apply_ocr=True)

def preprocess_data(examples):
    # Pega as imagens, palavras e caixas delimitadoras
    images = [Image.open(path).convert("RGB") for path in examples['image_path']]
    words = examples['words']
    boxes = examples['bboxes']
    word_labels = examples['ner_tags']

    # Usa o processador para tokenizar as palavras e preparar as imagens
    encoded_inputs = processor(images, words, boxes=boxes, word_labels=word_labels,
                               padding="max_length", truncation=True)

    return encoded_inputs

# Aplicamos a função de pré-processamento a todo o dataset
processed_train_dataset = train_dataset.map(preprocess_data, batched=True, remove_columns=train_dataset.column_names, features=train_dataset.features)
processed_eval_dataset = eval_dataset.map(preprocess_data, batched=True, remove_columns=eval_dataset.column_names, features=eval_dataset.features)

# Define o formato para PyTorch
processed_train_dataset.set_format(type="torch")
processed_eval_dataset.set_format(type="torch")

print("\nPré-processamento concluído!")
print(f"Exemplo de uma amostra processada (chaves): {processed_train_dataset[0].keys()}")


# ==============================================================================
# PASSO 3: FINE-TUNING DO MODELO
# ==============================================================================
print("\n>>> Configurando e iniciando o Fine-Tuning...")
from transformers import LayoutLMv3ForTokenClassification, TrainingArguments, Trainer
import evaluate
import numpy as np

# Carregamos o modelo pré-treinado
model = LayoutLMv3ForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id
)

# Carregamos a métrica 'seqeval'
metric = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Removemos os tokens especiais
    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id2label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

# Argumentos de treinamento
training_args = TrainingArguments(
    output_dir="meu-modelo-layoutlmv3-notas-fiscais",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=3e-5,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    push_to_hub=False,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=processed_train_dataset,
    eval_dataset=processed_eval_dataset,
    compute_metrics=compute_metrics,
)

# Inicia o treinamento
print("\nIniciando o treinamento. Isso levará alguns minutos...")
trainer.train()

# ==============================================================================
# PASSO 4: SALVANDO O MODELO FINAL
# ==============================================================================
print("\n>>> Salvando o modelo treinado localmente...")
trainer.save_model("modelo-notas-fiscais-final")
processor.save_pretrained("modelo-notas-fiscais-final")
print("Modelo salvo na pasta 'modelo-notas-fiscais-final'!")


# ==============================================================================
# PASSO 5: INFERÊNCIA - USANDO O MODELO PARA EXTRAIR INFORMAÇÕES
# ==============================================================================
print("\n>>> Executando inferência em uma nova imagem...")
from transformers import pipeline

pipe = pipeline("document-question-answering", model="./modelo-notas-fiscais-final")

image_to_test = Image.open(eval_dataset[0]['image_path']).convert("RGB")

from transformers import AutoModelForTokenClassification
import torch

model_final = AutoModelForTokenClassification.from_pretrained("./modelo-notas-fiscais-final")
processor_final = AutoProcessor.from_pretrained("./modelo-notas-fiscais-final")

def extract_entities(image, model, processor):
    encoding = processor(image, return_tensors="pt")
    for k,v in encoding.items():
      encoding[k] = v.to(model.device)

    with torch.no_grad():
      outputs = model(**encoding)

    logits = outputs.logits
    predictions = logits.argmax(-1).squeeze().tolist()
    boxes = encoding["bbox"].squeeze().tolist()

    results = []
    for pred, box in zip(predictions, boxes):
        if pred != label2id['O']:
            results.append({
                "label": model.config.id2label[pred],
                "box": box
            })
    return results

extracted_data = extract_entities(image_to_test, model_final, processor_final)

print("\n--- DADOS EXTRAÍDOS ---")
print(extracted_data)

from PIL import ImageDraw

draw = ImageDraw.Draw(image_to_test)
for data in extracted_data:
    box = [int(coord) for coord in data['box']]
    draw.rectangle(box, outline="red", width=2)

print("\nExibindo imagem com as entidades detectadas (salva como 'resultado.png')...")
image_to_test.save("resultado.png")

from google.colab.patches import cv2_imshow
import cv2
img_display = cv2.imread("resultado.png")
cv2_imshow(img_display)
